Pharma Dataset - I&A

In [1]:
import pandas as pd
import numpy as np
pharma_data = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Training_set_advc.csv')

In [2]:
pharma_data.head()

,ID_Patient_Care_Situation,Diagnosed_Condition,Patient_ID,Treated_with_drugs,Patient_Age,Patient_Body_Mass_Index,Patient_Smoker,Patient_Rural_Urban,Patient_mental_condition,A,B,C,D,E,F,Z,Number_of_prev_cond,Survived_1_year
0,16201,47,8433,DX2,60,21.655523,NO,URBAN,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1
1,9421,3,2972,DX6,2,28.852743,NO,RURAL,Stable,1.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,0
2,16205,7,8608,Dx6,20,26.179725,NO,RURAL,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1
3,5582,31,10074,dx6,8,22.638945,NO,RURAL,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0
4,20880,43,7462,dx1,53,21.326131,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [3]:
pharma_data.isnull().sum()

ID_Patient_Care_Situation       0
Diagnosed_Condition             0
Patient_ID                      0
Treated_with_drugs              8
Patient_Age                     0
Patient_Body_Mass_Index         0
Patient_Smoker                  0
Patient_Rural_Urban             0
Patient_mental_condition        0
A                            1356
B                            1356
C                            1356
D                            1356
E                            1356
F                            1356
Z                            1356
Number_of_prev_cond          1356
Survived_1_year                 0
dtype: int64

In [4]:
pharma_data['A'].fillna(pharma_data['A'].mode()[0], inplace=True)
pharma_data['B'].fillna(pharma_data['B'].mode()[0], inplace=True)
pharma_data['C'].fillna(pharma_data['C'].mode()[0], inplace=True)
pharma_data['D'].fillna(pharma_data['D'].mode()[0], inplace=True)
pharma_data['E'].fillna(pharma_data['E'].mode()[0], inplace=True)
pharma_data['F'].fillna(pharma_data['F'].mode()[0], inplace=True)
pharma_data['Z'].fillna(pharma_data['Z'].mode()[0], inplace=True)

In [5]:
pharma_data['Number_of_prev_cond'] = pharma_data['Number_of_prev_cond'].replace(np.nan, 7)

In [6]:
pharma_data['Treated_with_drugs'] = pharma_data['Treated_with_drugs'].str.rstrip()
pharma_data['Treated_with_drugs'].fillna(pharma_data['Treated_with_drugs'].mode()[0], inplace=True)
pharma_data['Treated_with_drugs'] = pharma_data['Treated_with_drugs'].str.lower()
pharma_data['Patient_Smoker'] = pharma_data['Patient_Smoker'].str.lower()
pharma_data['Patient_Rural_Urban'] = pharma_data['Patient_Rural_Urban'].str.lower()
pharma_data['Patient_mental_condition'] = pharma_data['Patient_mental_condition'].str.lower()
pharma_data['Patient_Smoker'] = pharma_data['Patient_Smoker'].replace(['no '], 'no')
pharma_data['Patient_Smoker'] = pharma_data['Patient_Smoker'].replace(['yes ','yess','yess '], 'yes')
pharma_data['Patient_Smoker'] = pharma_data['Patient_Smoker'].replace(['cannot say '], 'cannot say')
pharma_data = pd.concat([pharma_data, pd.get_dummies(pharma_data['Treated_with_drugs'].str.get_dummies(' '))], axis=1)
pharma_data = pd.concat([pharma_data, pd.get_dummies(pharma_data['Patient_Smoker'])], axis=1)
pharma_data = pd.concat([pharma_data, pd.get_dummies(pharma_data['Patient_Rural_Urban'])], axis=1)
pharma_data = pd.concat([pharma_data, pd.get_dummies(pharma_data['Patient_mental_condition'])], axis=1)

In [7]:
pharma_data = pharma_data.drop(columns=['Patient_Smoker', 'Patient_Rural_Urban','Patient_mental_condition','Treated_with_drugs','Patient_ID','cannot say'])

In [10]:
x = pharma_data.drop('Survived_1_year', axis = 1)
y = pharma_data['Survived_1_year']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=43)

In [11]:
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error

In [22]:
xgb_model = xgb.XGBClassifier(max_depth=6,n_estimators=200,learning_rate=0.1).fit(X_train, y_train)
predictions = xgb_model.predict(X_test)
actuals = y_test
print(confusion_matrix(actuals, predictions))

[[2292  696]
 [ 581 4708]]


In [23]:
print(mean_squared_error(actuals, predictions))

0.15428295276066206


In [15]:
test_new = pd.read_csv('https://raw.githubusercontent.com/dphi-official/Datasets/master/pharma_data/Testing_set_advc.csv')

In [17]:
test_new['Treated_with_drugs'] = test_new['Treated_with_drugs'].str.rstrip()
test_new['Treated_with_drugs'].fillna(test_new['Treated_with_drugs'].mode()[0], inplace=True)
test_new['Treated_with_drugs'] = test_new['Treated_with_drugs'].str.lower()
test_new['Patient_Smoker'] = test_new['Patient_Smoker'].str.lower()
test_new['Patient_Rural_Urban'] = test_new['Patient_Rural_Urban'].str.lower()
test_new['Patient_mental_condition'] = test_new['Patient_mental_condition'].str.lower()
test_new['Patient_Smoker'] = test_new['Patient_Smoker'].replace(['no '], 'no')
test_new['Patient_Smoker'] = test_new['Patient_Smoker'].replace(['yes ','yess','yess '], 'yes')
test_new['Patient_Smoker'] = test_new['Patient_Smoker'].replace(['cannot say '], 'cannot say')
test_new = pd.concat([test_new, pd.get_dummies(test_new['Treated_with_drugs'].str.get_dummies(' '))], axis=1)
test_new = pd.concat([test_new, pd.get_dummies(test_new['Patient_Smoker'])], axis=1)
test_new = pd.concat([test_new, pd.get_dummies(test_new['Patient_Rural_Urban'])], axis=1)
test_new = pd.concat([test_new, pd.get_dummies(test_new['Patient_mental_condition'])], axis=1)

In [18]:
test_new = test_new.drop(columns=['Patient_Smoker', 'Patient_Rural_Urban','Patient_mental_condition','Treated_with_drugs','Patient_ID'])

In [24]:
predictions = xgb_model.predict(test_new)

## How to save prediciton results locally via jupyter notebook?
If you are working on Jupyter notebook, execute below block of codes. A file named 'prediction_results.csv' will be created in your current working directory.

In [25]:
res = pd.DataFrame(predictions)
res.index = test_new.index # its important for comparison
res.columns = ["prediction"]
res.to_csv("C:/Users/kc/Desktop/dphibootcamp/a3/prediction_results_HP.csv")      # the csv file will be saved locally on the same location where this notebook is located.

## How to save prediction results locally via colab notebook?
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [201]:
# To create Dataframe of predicted value with particular respective index
res = pd.DataFrame(predictions)
res.index = test_new.index # its important for comparison
res.columns = ["prediction"]

# To download the csv file locally
from google.colab import files
res.to_csv('prediction_results_HP.csv')         
files.download('prediction_results_HP.csv')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>